In [ ]:
import pandas as pd
import os

import IPython.display as display
from PIL import Image
from PIL.ImageDraw import Draw

In [ ]:
from skimage.feature import hog
from skimage import color
from skimage.transform import resize

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix
import numpy as np

In [ ]:
data_dir = '../input/gtsrb-german-traffic-sign'
df = pd.read_csv(os.path.join(data_dir, 'Train.csv'))
df.head(5)

In [ ]:
for index, row in df.sample(frac=1)[:5].iterrows():
    image = Image.open(os.path.join(data_dir, row.Path))
    draw = Draw(image)
#     draw.rectangle([row.Roi.x1, row.Roi.y1, row.Roi.x2, row.Roi.y2], outline='#00FF00', width=3)
    display.display(image)
    print('category:', row.ClassId)

In [ ]:
def view_image(row):
    image = Image.open(os.path.join(data_dir, row.Path))
    draw = Draw(image)
#     draw.rectangle([row.Roi.x1, row.Roi.y1, row.Roi.x2, row.Roi.y2], outline='#00FF00', width=3)
    display.display(image)
    print('category:', row.ClassId)


In [ ]:
data = df.loc[(df['ClassId'] == 33) | (df['ClassId'] == 34)]
data

In [ ]:
unk = df.loc[(df['ClassId'] != 33) & (df['ClassId'] != 34)].sample(500)
unk.head(5)

In [ ]:
data = data.append(unk)
len(data)

In [ ]:
view_image(data.iloc[1000])

In [ ]:
view_image(data.iloc[1300])

In [ ]:
view_image(data.iloc[10])

In [ ]:
from sklearn.model_selection import train_test_split
#train test split
train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
len(test)

In [ ]:
import cv2


In [ ]:
x_train = [cv2.imread(os.path.join(data_dir, path), cv2.IMREAD_GRAYSCALE) for path in train['Path']]

In [ ]:
x_train[0]

In [ ]:
y_train = train['ClassId'].copy()


In [ ]:
y_train[(y_train != 33) & (y_train != 34)] = 0

In [ ]:
ppc = 32
hog_images = []
hog_features = []
for i in range(len(x_train)):
    image = x_train[i]
    image = resize(image, (128, 128), anti_aliasing=True)
    fd= hog(image, orientations=4, pixels_per_cell=(ppc,ppc),cells_per_block=(2, 2),block_norm= 'L2')
#     hog_images.append(hog_image)
    hog_features.append(fd)

In [ ]:
clf = svm.SVC()
# hog_features = np.array(hog_features)
# train_frame = np.hstack((hog_features,y_train))
# np.random.shuffle(train_frame)

In [ ]:
clf.fit(hog_features, y_train)

In [ ]:
y_pred = clf.predict(hog_features)

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
x_test = [cv2.imread(os.path.join(data_dir, path), cv2.IMREAD_GRAYSCALE) for path in test['Path']]
y_test = test['ClassId'].copy()

In [ ]:
y_test[(y_test != 33) & (y_test != 34)] = 0

In [ ]:
ppc = 32
test_features = []
for i in range(len(x_test)):
    image = x_test[i]
    image = resize(image, (128, 128), anti_aliasing=True)
    fd= hog(image, orientations=4, pixels_per_cell=(ppc,ppc),cells_per_block=(2, 2),block_norm= 'L2')
#     hog_images.append(hog_image)
    test_features.append(fd)

In [ ]:
y_pred = clf.predict(test_features)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
np.where(y_test != y_pred)

In [ ]:
view_image(unk.iloc[291])

In [ ]:
view_image(unk.iloc[53])